# Nhanh Bills - Local Testing

Notebook này cho phép chạy Nhanh Bills pipeline từ local, không cần gọi Cloud Run.

**⚠️ QUAN TRỌNG - Đồng bộ với CloudRun:**
- Notebook này import trực tiếp từ code chính (`src/features/nhanh/bills/`)
- Khi code trên CloudRun thay đổi, notebook tự động sử dụng code mới
- Không cần copy/paste code - chỉ cần chạy lại notebook sau khi code thay đổi

**Flow:**
1. Extract: Lấy dữ liệu bills từ Nhanh API
2. Load: Upload lên GCS (Bronze layer) và setup External Tables
3. Transform: Transform từ Raw → Clean (Flattened) tables

**Sử dụng khi:**
- Test pipeline trước khi deploy
- Debug issues
- Chạy sync thủ công cho date range cụ thể
- Test với credentials local


In [ ]:
import os
import sys
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))
sys.path.insert(0, project_root)

# Load environment variables
load_dotenv()

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")


In [ ]:
# Import các module từ code chính
# Lưu ý: Notebook import trực tiếp từ src/features/nhanh/bills/ nên luôn đồng bộ với code CloudRun
from src.features.nhanh.bills.components.extractor import BillExtractor
from src.features.nhanh.bills.components.transformer import BillTransformer
from src.features.nhanh.bills.components.loader import BillLoader
from src.features.nhanh.bills.pipeline import BillPipeline
from src.config import settings
import logging

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Đã import tất cả modules thành công")


## Cấu hình

Load cấu hình từ environment variables và credential files.

**Lưu ý về Credentials:**
- Nhanh API credentials được lấy từ Secret Manager hoặc environment variables
- BigQuery credentials: Sử dụng Application Default Credentials (ADC) hoặc service account key
- GCS bucket: Cấu hình trong `BRONZE_BUCKET` environment variable


In [ ]:
# ⚠️ QUAN TRỌNG: Reload modules sau khi sửa code
# Chạy cell này trước khi chạy pipeline để đảm bảo code mới nhất được sử dụng
import importlib
import src.features.nhanh.bills.components.extractor
import src.features.nhanh.bills.components.transformer
import src.features.nhanh.bills.components.loader
import src.features.nhanh.bills.pipeline

# Reload modules
importlib.reload(src.features.nhanh.bills.components.extractor)
importlib.reload(src.features.nhanh.bills.components.transformer)
importlib.reload(src.features.nhanh.bills.components.loader)
importlib.reload(src.features.nhanh.bills.pipeline)

# Re-import sau khi reload
from src.features.nhanh.bills.components.extractor import BillExtractor
from src.features.nhanh.bills.components.transformer import BillTransformer
from src.features.nhanh.bills.components.loader import BillLoader
from src.features.nhanh.bills.pipeline import BillPipeline

# Re-initialize components với code mới
extractor = BillExtractor()
transformer = BillTransformer()
loader = BillLoader()
pipeline = BillPipeline()

print("✅ Đã reload modules và khởi tạo lại components thành công")


In [ ]:
# Kiểm tra cấu hình
print(f"GCP Project: {settings.gcp_project}")
print(f"GCP Region: {settings.gcp_region}")
print(f"Bronze Bucket: {settings.bronze_bucket}")
print(f"Bronze Dataset: {settings.bronze_dataset}")
print(f"Target Dataset: {settings.target_dataset}")
print(f"Nhanh API Base URL: {settings.nhanh_api_base_url}")

# Kiểm tra credentials (nếu có trong env)
nhanh_creds_available = (
    os.getenv("NHANH_APP_ID") or 
    os.getenv("NHANH_BUSINESS_ID") or 
    os.getenv("NHANH_ACCESS_TOKEN")
)
print(f"\nNhanh Credentials từ env: {'✅ Có' if nhanh_creds_available else '⚠️ Không có (sẽ dùng Secret Manager)'}")


## Khởi tạo Components

Khởi tạo extractor, transformer, loader và pipeline.


In [ ]:
# Khởi tạo các components
extractor = BillExtractor()
transformer = BillTransformer()
loader = BillLoader()

# Khởi tạo pipeline
pipeline = BillPipeline()

print("✅ Đã khởi tạo tất cả components thành công")


## Chạy Pipeline

Chạy pipeline với các options khác nhau.


### Option 1: Extract + Load (Bronze Layer)

Chạy Extract và Load cho date range cụ thể.


In [ ]:
# Extract + Load cho 30 ngày gần nhất
from_date = datetime.now() - timedelta(days=30)
to_date = datetime.now()

print("=" * 60)
print(f"Đang chạy Extract + Load Pipeline")
print(f"Date range: {from_date.strftime('%Y-%m-%d')} đến {to_date.strftime('%Y-%m-%d')}")
print("=" * 60)

result = pipeline.run_extract_load(
    from_date=from_date,
    to_date=to_date,
    process_by_day=True
)

print("\n" + "=" * 60)
print("Kết quả Pipeline:")
print("=" * 60)
for key, value in result.items():
    print(f"{key}: {value}")
print("=" * 60)


### Option 2: Transform (Raw → Clean)

Chạy Transform để flatten data từ External Tables sang Clean Tables.


In [ ]:
# Chạy Transform
print("=" * 60)
print("Đang chạy Transform Pipeline (Raw → Clean)")
print("=" * 60)

result = pipeline.run_transform()

print("\n" + "=" * 60)
print("Kết quả Transform:")
print("=" * 60)
for key, value in result.items():
    print(f"{key}: {value}")
print("=" * 60)


### Option 3: Full Pipeline (Extract + Load + Transform)

Chạy toàn bộ pipeline từ đầu đến cuối.


In [ ]:
# Full pipeline cho 30 ngày gần nhất
from_date = datetime.now() - timedelta(days=30)
to_date = datetime.now()

print("=" * 60)
print(f"Đang chạy Full Pipeline")
print(f"Date range: {from_date.strftime('%Y-%m-%d')} đến {to_date.strftime('%Y-%m-%d')}")
print("=" * 60)

result = pipeline.run_full_pipeline(
    from_date=from_date,
    to_date=to_date
)

print("\n" + "=" * 60)
print("Kết quả Full Pipeline:")
print("=" * 60)
for key, value in result.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key}: {value}")
print("=" * 60)


### Option 4: Chạy với Date Range Tùy Chỉnh

Chạy pipeline với ngày cụ thể.


In [ ]:
# # Tùy chọn: Chạy với date range tùy chỉnh
# custom_from_date = datetime(2025, 1, 1)
# custom_to_date = datetime(2025, 1, 15)

# print("=" * 60)
# print(f"Đang chạy Pipeline cho date range: {custom_from_date.strftime('%Y-%m-%d')} đến {custom_to_date.strftime('%Y-%m-%d')}")
# print("=" * 60)

# result = pipeline.run_extract_load(
#     from_date=custom_from_date,
#     to_date=custom_to_date,
#     process_by_day=True
# )

# print("\n" + "=" * 60)
# print("Kết quả Pipeline:")
# print("=" * 60)
# for key, value in result.items():
#     print(f"{key}: {value}")
# print("=" * 60)


## Test Từng Component Riêng Lẻ

Test từng component riêng lẻ để debug.


### Test Extractor

Test extractor để lấy dữ liệu từ Nhanh API.


In [ ]:
# # Test Extractor - Extract bills và products
# test_from_date = datetime.now() - timedelta(days=7)
# test_to_date = datetime.now()

# print(f"Đang test Extractor cho date range: {test_from_date.strftime('%Y-%m-%d')} đến {test_to_date.strftime('%Y-%m-%d')}")

# bills, products = extractor.extract_with_products(
#     from_date=test_from_date,
#     to_date=test_to_date,
#     process_by_day=True
# )

# print(f"\nKết quả extract:")
# print(f"  - Bills: {len(bills)} records")
# print(f"  - Products: {len(products)} records")
# if bills:
#     print(f"  - Sample bill columns: {list(bills[0].keys())[:10]}")
# if products:
#     print(f"  - Sample product columns: {list(products[0].keys())[:10]}")


### Test Loader

Test loader để upload lên GCS và setup External Tables.


In [ ]:
# # Test Loader - Upload và setup External Tables
# # Lưu ý: Cần có dữ liệu từ extractor trước
# from datetime import date

# test_date = date.today()
# test_bills = [{"id": "test", "code": "TEST001"}]  # Sample data
# test_products = [{"billId": "test", "productId": "prod1"}]  # Sample data

# print(f"Đang test Loader cho ngày: {test_date}")

# # Upload bills
# if test_bills:
#     bills_path = loader.load_bills(data=test_bills, partition_date=test_date)
#     print(f"Bills uploaded to: {bills_path}")

# # Upload products
# if test_products:
#     products_path = loader.load_bill_products(data=test_products, partition_date=test_date)
#     print(f"Products uploaded to: {products_path}")

# # Setup External Tables
# print("\nSetting up External Tables...")
# external_tables = loader.setup_external_tables()
# print(f"External Tables: {external_tables}")


### Test Transformer

Test transformer để transform data từ Raw sang Clean.


In [ ]:
# # Test Transformer - Transform từ Raw → Clean
# print("Đang test Transformer...")
# print("⚠️ Lưu ý: Cần có dữ liệu trong External Tables trước")

# result = transformer.transform_flatten()

# print(f"\nKết quả Transform:")
# for key, value in result.items():
#     print(f"  - {key}: {value}")
